Do clonality analysis in python

In [2]:
import pandas as pd
from itertools import combinations

def get_overlap_table(sample_vec, data):
    res_count = pd.DataFrame(data=None, index=sample_vec, columns=sample_vec)
    res_count_r = pd.DataFrame(data=None, index=sample_vec, columns=sample_vec)
    res_count_f = pd.DataFrame(data=None, index=sample_vec, columns=sample_vec)
    result = data.copy()
    for s1i, s2i in combinations(range(len(sample_vec)), 2):
        s1 = sample_vec[s1i]
        s2 = sample_vec[s2i]
        print(f"start processing: {s1} and {s2} pair")
        new = get_overlap(s1, s2, data)
        result = pd.concat([result, new], axis=1)
        
        coln = f"is_{s1}_{s2}"
        is_count = len(''.join(result[coln]))
        coln = f"u_{s1}_{s2}"
        res_count.iloc[s1i, s2i] = len(''.join(result[coln]))
        res_count_r.iloc[s1i, s2i] = res_count.iloc[s1i, s2i] / (res_count.iloc[s1i, s2i] + is_count)
        coln = f"u_{s1}_{s2}_f"
        res_count_f.iloc[s1i, s2i] = result[coln].sum()
        coln = f"u_{s2}_{s1}"
        res_count.iloc[s2i, s1i] = len(''.join(result[coln]))
        res_count_r.iloc[s2i, s1i] = res_count.iloc[s2i, s1i] / (res_count.iloc[s2i, s1i] + is_count)
        coln = f"u_{s2}_{s1}_f"
        res_count_f.iloc[s2i, s1i] = result[coln].sum()
    print("finished get_overlap_table")
    return {"res_count": res_count, "res_count_r": res_count_r, "res_count_f": res_count_f, "result": result}

def count_ele(vec1, vec2):
    return sum(1 for x in vec1 if x in vec2)

def get_overlap(s1, s2, data):
    is_s1_s2_vec = []
    u_s1_s2_vec = []
    u_s1_s2_f_vec = []
    u_s2_s1_vec = []
    u_s2_s1_f_vec = []
    
    for i, row in data.iterrows():
        #if i % 100000 == 0:
            #print(f"finished processing variant: {i}")
        vec1 = row[s1].split('/')
        #print("vec1", vec1)
        vec2 = row[s2].split('/')
        #print("vec2", vec2)
        
        is_s1_s2 = list(set(vec1) & set(vec2))
        is_s1_s2_vec.append(''.join(is_s1_s2))
        #print("is_s1_s2_vec", is_s1_s2_vec)
        
        u_s1_s2 = list(set(vec1) - set(vec2)) # get allele that is unique to s1
        u_s1_s2_vec.append(''.join(u_s1_s2))
        #print("u_s1_s2_vec", u_s1_s2_vec)
        
        u_s1_s2_f = count_ele(vec1, u_s1_s2)
        u_s1_s2_f_vec.append(u_s1_s2_f)
        #print("u_s1_s2_f_vec", u_s1_s2_f_vec)
        
        u_s2_s1 = list(set(vec2) - set(vec1))
        u_s2_s1_vec.append(''.join(u_s2_s1))
        #print("u_s2_s1_vec", u_s2_s1_vec)
        
        u_s2_s1_f = count_ele(vec2, u_s2_s1)
        u_s2_s1_f_vec.append(u_s2_s1_f)
        #print("u_s2_s1_f_vec", u_s2_s1_f_vec)
    
    df = pd.DataFrame({
        f"is_{s1}_{s2}": is_s1_s2_vec,
        f"u_{s1}_{s2}": u_s1_s2_vec,
        f"u_{s1}_{s2}_f": u_s1_s2_f_vec,
        f"u_{s2}_{s1}": u_s2_s1_vec,
        f"u_{s2}_{s1}_f": u_s2_s1_f_vec
    })
    return df

In [3]:
# data_name = "p4/bindedp4_set240924_DP2_INDEL"

# sample_vec = ["SM44B", "C1C5v2", "C1A10", "C1E4", "C2F9", 
#                  "SM60B", "C3D7v2", "C1C51", "C2H11",
#                  "C1G11", "C1A1", "C1A4"]

# "command" and "/" to comment multiple lines

data_name = "p4/bindedp4_set240924_wo_C1A4_DP5_INDEL"

sample_vec = ["SM44B", "C1C5v2", "C1A10", "C1E4", "C2F9", 
                "SM60B", "C3D7v2", "C1C51", "C2H11",
                "C1G11", "C1A1"]

In [4]:
data_csv = data_name + ".csv"
data = pd.read_csv(data_csv)

In [5]:
result_lis2 = get_overlap_table(sample_vec, data)

start processing: SM44B and C1C5v2 pair
start processing: SM44B and C1A10 pair
start processing: SM44B and C1E4 pair
start processing: SM44B and C2F9 pair


KeyboardInterrupt: 

In [72]:
res_count_all_p4 = result_lis2["res_count"]

In [73]:
res_count_all_p4

,SM44B,C1C5v2,C1A10,C1E4,C2F9,SM60B,C3D7v2,C1C51,C2H11,C1G11,C1A1
SM44B,NaN,851,783,742,843,27941,28037,27922,27812,22977,23834
C1C5v2,1454,NaN,1380,1249,1543,28389,28325,28368,28344,23508,24205
C1A10,879,873,NaN,735,902,28061,28124,28037,27927,23098,23951
C1E4,1115,1019,1012,NaN,1191,28179,28219,28116,28052,23186,24043
C2F9,782,879,745,757,NaN,27967,28061,27947,27842,23016,23877
SM60B,28556,28401,28580,28421,28643,NaN,1033,785,625,23192,24204
C3D7v2,28778,28463,28769,28587,28863,1159,NaN,1223,1074,23337,24228
C1C51,28566,28409,28585,28387,28652,814,1126,NaN,729,23174,24202
C2H11,28828,28757,28847,28695,28919,1026,1349,1101,NaN,23546,24601
C1G11,44280,44208,44305,44116,44380,43880,43899,43833,43833,NaN,8449


In [74]:
res_count_csv = data_name + "_res_count.csv"
res_count_all_p4.to_csv(res_count_csv)

In [75]:
res_count_f_all_p4 = result_lis2["res_count_f"]

In [76]:
res_count_f_csv = data_name + "_res_count_f.csv"
res_count_f_all_p4.to_csv(res_count_f_csv)

In [77]:
res_count_r_all_p4 = result_lis2["res_count_r"]

In [78]:
res_count_r_csv = data_name + "_res_count_r.csv"
res_count_r_all_p4.to_csv(res_count_r_csv)